In [ ]:
import pandas as pd
from urllib.parse import urlparse
import requests

In [ ]:
def retrieval_eval(query, gold_urls):
  retrieval_1 = 0
  retrieval_5 = 0
  retrieval_10 = 0
  retrieval_20 = 0
  retrieval_50 = 0
  mrr = 0
  reciprocal_ranks = []

  for i in range(0,len(query)):
    added = False
    cand = query[i]
    gold_url = urlparse(gold_urls[i])
    gold_url_netloc = gold_url[1]
    gold_url_path = gold_url[2]
    if gold_url_path[-1]=='/':
      gold_url_path = gold_url_path[:-1]
    
    params = {
            'query': cand.strip(),
            'pageSize': '50',
            'key': '<your_key>',
        }
    try:
        response_dict = requests.get(
            'https://factchecktools.googleapis.com/v1alpha1/claims:search',
            params=params).json()
    except Exception as e:
      response_dict = {}
      print("Google API is not working")
      break

    if 'claims' in response_dict.keys():
      for i in range(0, len(response_dict["claims"])):
        claim = response_dict['claims'][i]
        ret_url = urlparse(claim['claimReview'][0]['url'])
        ret_url_netloc = ret_url[1]
        ret_url_path = ret_url[2]
        if ret_url_path[-1]=='/':
          ret_url_path = ret_url_path[:-1]

        if(ret_url_netloc == gold_url_netloc and ret_url_path == gold_url_path):
          added = True
          reciprocal_ranks.append(1/(i+1))
          if i<1:
            retrieval_1 +=1

          if i<5:
            retrieval_5 +=1

          if i<10:
            retrieval_10 +=1  

          if i<20:
            retrieval_20 +=1

          retrieval_50 +=1
          break
    
    if added == False:
      reciprocal_ranks.append(0)

  if len(reciprocal_ranks) >0:
    mrr = sum(reciprocal_ranks)/len(reciprocal_ranks)

  return { 
      'retrieval_1': (retrieval_1*100)/len(query),
      'retrieval_5': (retrieval_5*100)/len(query),
      'retrieval_10': (retrieval_10*100)/len(query),
      'retrieval_20': (retrieval_20*100)/len(query),
      'retrieval_50': (retrieval_50*100)/len(query),
      'mrr': mrr,
      }

In [ ]:
base_file = 'summ_data_eng_eng_raw.csv'
pred_filenames = ['predictions.xlsx', 'predictions(1).xlsx', 'predictions(2).xlsx', 'predictions(3).xlsx', 'predictions(4).xlsx']

In [ ]:
summ_data_eng_eng_raw = pd.read_csv(base_file, header=0, index_col=0)
for file in pred_filenames:
  preds = pd.read_csv(file, header=0, index_col=0)
  gold_urls = []
  for index, row in preds.iterrows():
    gold_urls.append(summ_data_eng_eng_raw.loc[row['ID']]['evidence_url'])
  print("{} Retrieval = {}%".format(file, retrieval_eval(list(preds['Generated Text']), gold_urls)))

predictions.xlsx Retrieval = {'retrieval_1': 17.54385964912281, 'retrieval_5': 26.31578947368421, 'retrieval_10': 27.19298245614035, 'retrieval_20': 27.19298245614035, 'retrieval_50': 27.19298245614035, 'mrr': 0.21661793372319688}%
predictions(1).xlsx Retrieval = {'retrieval_1': 21.92982456140351, 'retrieval_5': 29.82456140350877, 'retrieval_10': 29.82456140350877, 'retrieval_20': 29.82456140350877, 'retrieval_50': 29.82456140350877, 'mrr': 0.2529239766081871}%
predictions(2).xlsx Retrieval = {'retrieval_1': 18.58407079646018, 'retrieval_5': 27.43362831858407, 'retrieval_10': 29.20353982300885, 'retrieval_20': 29.20353982300885, 'retrieval_50': 29.20353982300885, 'mrr': 0.22448377581120946}%
predictions(3).xlsx Retrieval = {'retrieval_1': 19.469026548672566, 'retrieval_5': 26.548672566371682, 'retrieval_10': 26.548672566371682, 'retrieval_20': 26.548672566371682, 'retrieval_50': 26.548672566371682, 'mrr': 0.22640117994100298}%
predictions(4).xlsx Retrieval = {'retrieval_1': 20.35398230